In [2]:
from database.mainstbets import MainStBets
import pandas as pd
from tqdm import tqdm
from datetime import datetime,timedelta
import matplotlib.pyplot as plt

In [5]:
market = MainStBets()

In [6]:
market.connect()
sp500 = market.retrieve("sp500")
prices = market.retrieve("prices")
market.disconnect()

In [14]:
sp500.columns

Index(['_id', 'Symbol', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK',
       'Founded'],
      dtype='object')

In [7]:
prices["date"] = pd.to_datetime(prices["date"])

In [8]:
prices.columns

Index(['_id', 'date', 'close', 'high', 'low', 'open', 'volume', 'adjClose',
       'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor',
       'ticker'],
      dtype='object')

In [9]:
final = []
for ticker in tqdm(sp500["Symbol"]):
    ticker_data = prices[prices["ticker"]==ticker]
    ticker_data["rolling"] = ticker_data["adjClose"].rolling(window=100).mean()
    ticker_data["gain"] = (ticker_data["adjClose"] - ticker_data["rolling"]) / ticker_data["rolling"]
    final.append(ticker_data)

  0%|                                                  | 0/505 [00:00<?, ?it/s]C:\Users\ericj\AppData\Local\Temp/ipykernel_17812/1279899860.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data["rolling"] = ticker_data["adjClose"].rolling(window=100).mean()
C:\Users\ericj\AppData\Local\Temp/ipykernel_17812/1279899860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data["gain"] = (ticker_data["adjClose"] - ticker_data["rolling"]) / ticker_data["rolling"]
100%|████████████████████████████

In [10]:
sim = pd.concat(final)
sim.dropna(inplace=True)

In [11]:
sim["year"] = [x.year for x in sim["date"]]

In [15]:
today = sim[sim["date"]==sim["date"].max()][["date","adjClose","ticker","gain","rolling"]].sort_values("gain").head(20).merge(sp500[["Symbol","Security","GICS Sector"]].rename(columns={"Symbol":"ticker"}),on="ticker")

In [19]:
recent = today[["date","GICS Sector","Security","ticker","rolling","adjClose","gain"]]

In [20]:
recent

,date,GICS Sector,Security,ticker,rolling,adjClose,gain
0,2021-08-24 00:00:00+00:00,Consumer Discretionary,Las Vegas Sands,LVS,52.696900,42.72,-0.189326
1,2021-08-24 00:00:00+00:00,Materials,FMC Corporation,FMC,109.958259,90.75,-0.174687
2,2021-08-24 00:00:00+00:00,Information Technology,IPG Photonics,IPGP,204.241800,170.38,-0.165793
3,2021-08-24 00:00:00+00:00,Consumer Discretionary,Wynn Resorts,WYNN,117.916400,99.44,-0.156691
4,2021-08-24 00:00:00+00:00,Consumer Discretionary,General Motors,GM,57.362200,49.57,-0.135842
5,2021-08-24 00:00:00+00:00,Information Technology,Global Payments,GPN,193.508289,167.30,-0.135438
6,2021-08-24 00:00:00+00:00,Energy,Hess Corporation,HES,78.578755,68.19,-0.132208
7,2021-08-24 00:00:00+00:00,Information Technology,Citrix Systems,CTXS,118.171910,102.59,-0.131858
8,2021-08-24 00:00:00+00:00,Industrials,Nielsen Holdings,NLSN,25.064792,21.80,-0.130254
9,2021-08-24 00:00:00+00:00,Consumer Staples,Lamb Weston,LW,76.486058,66.63,-0.128861


In [17]:
full = sim[["date","adjClose","ticker","gain","rolling"]].merge(sp500[["Symbol","Security","GICS Sector"]].rename(columns={"Symbol":"ticker"}),on="ticker")[["date","GICS Sector","Security","ticker","rolling","adjClose","gain"]]

In [21]:
market.connect()
market.store("full",full)
market.store("recent",recent)
market.disconnect()